# AgentCore エンドツーエンド クリーンアップ

このノートブックは、AgentCore エンドツーエンドチュートリアル中に作成されたすべてのリソースの包括的なクリーンアッププロセスを提供します。

## 概要

このクリーンアッププロセスは以下を削除します：
- **Memory**: AgentCore Memory リソースと保存されたデータ
- **Runtime**: エージェント Runtime インスタンスと ECR リポジトリ
- **セキュリティ**: 実行ロール、および認可プロバイダーリソース
- **オブザーバビリティ**: CloudWatch ロググループとストリーム
- **ローカルファイル**: 生成された設定ファイルとコードファイル

**重要**: このクリーンアップは元に戻せません。続行する前に、重要なデータを保存したことを確認してください（必要な場合）。

---

## ステップ 1: 必要な依存関係のインポート

クリーンアッププロセスに必要なすべてのモジュールとヘルパー関数を読み込みます。

In [ ]:
import json

from lab_helpers.lab2_memory import REGION
from lab_helpers.utils import (
    delete_agentcore_runtime_execution_role,
    delete_ssm_parameter,
    cleanup_cognito_resources,
    get_customer_support_secret,
    delete_customer_support_secret,
    agentcore_memory_cleanup,
    gateway_target_cleanup,
    runtime_resource_cleanup,
    delete_observability_resources,
    local_file_cleanup,
    get_ssm_parameter,
)

print("✅ Dependencies imported successfully")
print(f"🌍 Working in region: {REGION}")

## ステップ 2: Memory リソースのクリーンアップ

AgentCore Memory リソースと関連データを削除します。

In [ ]:
print("🧠 Starting Memory cleanup...")
agentcore_memory_cleanup(get_ssm_parameter("/app/customersupport/agentcore/memory_id"))

## ステップ 3: Runtime リソースのクリーンアップ

AgentCore Runtime、ECR リポジトリ、および関連する AWS リソースを削除します。

In [ ]:
print("🚀 Starting Runtime cleanup...")
runtime_resource_cleanup(
    get_ssm_parameter("/app/customersupport/agentcore/runtime_arn")
)

## ステップ 4: Gateway リソースのクリーンアップ
ターゲット、Gateway を削除します。

In [ ]:
print("⚙️ Starting Gateway Cleanup...")
gateway_target_cleanup(get_ssm_parameter("/app/customersupport/agentcore/gateway_id"))

## ステップ 5: セキュリティリソースのクリーンアップ

実行ロールと認証リソースを削除します。

In [ ]:
print("🛡️  Starting Security cleanup...")
try:
    # bedrock_client = boto3.client("bedrock", region_name=REGION)

    # Delete execution role
    print("  🗑️  Deleting AgentCore Runtime execution role...")
    delete_agentcore_runtime_execution_role()
    print("  ✅ Execution role deleted")

    # Delete SSM parameter
    print("  🗑️  Deleting SSM parameter...")
    delete_ssm_parameter("/app/customersupport/agentcore/runtime_arn")
    print("  ✅ SSM parameter deleted")

    # Clean up Cognito and secrets
    print("  🗑️  Cleaning up Cognito resources...")
    cs = json.loads(get_customer_support_secret())
    cleanup_cognito_resources(cs["pool_id"])
    print("  ✅ Cognito resources cleaned up")

    print("  🗑️  Deleting customer support secret...")
    delete_customer_support_secret()
    print("  ✅ Customer support secret deleted")

except Exception as e:
    print(f"  ⚠️  Error during security cleanup: {e}")

## ステップ 6: ローカルファイルのクリーンアップ

ローカルディレクトリから生成された設定ファイルとコードファイルを削除します。

In [ ]:
print("📁 Starting Local Files cleanup...")
local_file_cleanup()

## ステップ 7: オブザーバビリティリソースのクリーンアップ

エージェント監視に使用された CloudWatch ロググループとストリームを削除します。

In [ ]:
print("📊 Starting Observability cleanup...")

delete_observability_resources()

## クリーンアップ完了！

すべての AgentCore リソースがクリーンアップされました。削除されたものの概要は以下の通りです：

In [ ]:
print("\n" + "=" * 60)
print("🧹 CLEANUP COMPLETED SUCCESSFULLY! 🧹")
print("=" * 60)
print()
print("📋 Resources cleaned up:")
print("  🧠 Memory: AgentCore Memory resources and data")
print("  🚀 Runtime: Agent runtime and ECR repository")
print("  🛡️ Security: Roles, and SSM secrets")
print("  📊 Observability: CloudWatch logs")
print("  📁 Files: Local configuration files")
print()
print("✨ Your AWS account is now clean and ready for new experiments!")
print("\nThank you for completing the AgentCore End-to-End tutorial! 🚀")